<a href="https://colab.research.google.com/github/pedrodeparis/Recommendation-Systems/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_networkx_Pinterest_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install networkx

In [4]:
import os
import pandas as pd
from google.colab import files
from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['007efd', 'FFC000', '#303030'])

In [5]:
%%time
_ = files.upload()

Saving pinterest.parquet to pinterest.parquet
CPU times: user 660 ms, sys: 96.1 ms, total: 756 ms
Wall time: 48.1 s


In [6]:
filepath = './pinterest.parquet'

df = pd.read_parquet(filepath)
df.tail()

,user_id,item_id,rating
1445617,U-55186,I-5448,1
1445618,U-55186,I-4615,1
1445619,U-55186,I-5346,1
1445620,U-55186,I-2803,1
1445621,U-55186,I-4207,1


In [7]:
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()
n_ratings = df[['user_id','item_id']].drop_duplicates().shape[0]

print (f'#Usuários: {n_users}')
print (f'#Itens: {n_items}')
print (f'#Avaliações: {n_ratings}')

#Usuários: 55187
#Itens: 9911
#Avaliações: 1408394


Criando grafo bipartido com nós do tipo item e do tipo usuário


Arestas entre um usuário e todos os itens salvos

In [14]:
G = nx.Graph()
G.add_nodes_from(df['item_id'].unique(), node_type='item')
G.add_nodes_from(df['user_id'].unique(), node_type='user')
G.add_weighted_edges_from(df[['user_id', 'item_id', 'rating']].values)

In [15]:
G

Analisando os grafos

In [16]:
G.number_of_nodes()

65098

In [17]:
G.number_of_edges()

1408394

Construindo a recomendação a partir do grafo.

In [20]:
item_id = df['item_id'][0]
neighbors = G.neighbors(item_id)
list(neighbors)[:5]

['U-0', 'U-56', 'U-347', 'U-365', 'U-454']

In [21]:
user_id = df['user_id'][0]
neighbors = G.neighbors(user_id)
list(neighbors)[:5]


['I-2', 'I-3', 'I-4', 'I-5', 'I-6']

Relação de quem viu algo tb viu algo.
  Escolhendo item-alvo

  Obtenção dos vizinhos do item-alvo (salvo pelos usuários)
  
  Vizinhos dos usuários que viram o item-alvo

In [22]:
item_id = 'I-5448'
neighbors = G.neighbors(item_id)

neighbor_consumed_items = []
for user_id in neighbors:
    user_consumed_items = G.neighbors(user_id)
    neighbor_consumed_items += list(user_consumed_items)

print ('Quem viu {} também viu: {}'.format(item_id, ','.join(neighbor_consumed_items[:20])))

Quem viu I-5448 também viu: I-5186,I-4665,I-2937,I-1751,I-3010,I-5445,I-5446,I-5447,I-5444,I-5237,I-356,I-4139,I-4020,I-5449,I-3979,I-2991,I-4021,I-5450,I-3104,I-5448


In [ ]:
consumed_items_count = Counter(neighbor_consumed_items, ascending=True)
consumed_items_count

In [25]:
def recommend_neighbor_items(G:nx.Graph, target_id, n=10):
    # Validando tipo do nó
    node_type = nx.get_node_attributes(G, 'node_type')[target_id]
    if node_type != 'item':
        raise ValueError('Node is not of item type.')

    # Analisando consumo dos usuários vizinhos
    neighbor_consumed_items = []
    for user_id in G.neighbors(target_id):
        user_consumed_items = G.neighbors(user_id)
        neighbor_consumed_items +=list(user_consumed_items)

    # Contabilizando itens consumidos pelos vizinhos
    consumed_items_count = Counter(neighbor_consumed_items)

    # Criando dataframe
    df_neighbors = pd.DataFrame(zip(consumed_items_count.keys(), consumed_items_count.values()))
    df_neighbors.columns = ['item_id', 'score']
    df_neighbors = df_neighbors.sort_values(by='score', ascending=False).set_index('item_id')
    return df_neighbors.head(n)

recommend_neighbor_items(G, 'I-5185', n=10)

,score
item_id,
I-5185,144
I-6618,28
I-3705,26
I-5942,24
I-5941,23
I-6080,23
I-4175,21
I-4003,21
I-6594,21


In [26]:
df.groupby('item_id')['user_id'].count().sort_values(ascending=False).head().index

Index(['I-487', 'I-442', 'I-773', 'I-3831', 'I-13'], dtype='object', name='item_id')

In [27]:
recommend_neighbor_items(G, 'I-487')

,score
item_id,
I-487,1544
I-773,170
I-817,153
I-931,150
I-877,139
I-495,130
I-1203,127
I-1198,127
I-485,125
